In [ ]:
import numpy as np
import keras
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import itertools
from keras.applications import vgg16
from keras.applications.vgg16 import preprocess_input

In [ ]:
train_path = r'train'
test_path = r'test'

In [ ]:
datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)
batch_size = 10

In [ ]:
# classes are automatically inferred from subdirectories
train_generator = ImageDataGenerator().flow_from_directory( train_path,
                                                            target_size=(224, 224),
                                                            batch_size=5,
                                                            class_mode='categorical',
                                                            shuffle=True)

test_generator = ImageDataGenerator().flow_from_directory( test_path,
                                                           target_size=(224, 224),
                                                           batch_size=5,
                                                           class_mode='categorical',
                                                           shuffle=True)

In [ ]:
# For visualising images with labels
def plots(ims, figsize=(16,16), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3): ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=8)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
img, labels = next(train_generator)
plots(img, titles=labels)

In [ ]:
%%time 
# Load the vgg16 model
vgg16_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224,3))

In [ ]:
# include_top=False removes last four layers flatten fc1 fc2 and predictions
vgg16_model_withTop = vgg16.VGG16(weights='imagenet', include_top=True, input_shape=(224, 224,3))

In [ ]:
vgg16_model_withTop.layers.pop()
vgg16_model_withTop.summary()

In [ ]:
# removing layers where training not required can be customised too
for layer in vgg16_model_withTop.layers:
    layer.trainable = False

In [ ]:
# updated way to do fine tuning in keras
base_model = vgg16_model_withTop
this_model = models.Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

In [ ]:
custom_Vgg16_Model = models.Sequential()
# making a sequential model out of vgg16 with top
custom_Vgg16_Model.add(this_model)
# add a 5 neuron dense layer in the end for classification of 5 categories
custom_Vgg16_Model.add(layers.Dense(2,activation='softmax'))

In [ ]:
custom_Vgg16_Model.summary()

In [ ]:
custom_Vgg16_Model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = custom_Vgg16_Model.fit_generator(train_generator, 
                                 steps_per_epoch=5000, 
                                 validation_data=test_generator, 
                                 validation_steps=10,
                                 epochs=5, 
                                 verbose=1)

In [ ]:
%time
test_imgs, test_labels = next(test_generator)
plots(test_imgs, titles=test_labels)

In [ ]:
from matplotlib import style
import seaborn as sns

%matplotlib inline
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

Salvar modelo

In [ ]:
custom_Vgg16_Model.save_weights("modeloPeles.h5")
print("Modelo salvo com sucesso!")

In [ ]:
# load weights into new model
custom_Vgg16_Model.load_weights("modeloPeles.h5")
print("Modelo carregado com sucesso!")

Predição individual

In [ ]:
# to make predictions
import numpy as np
from matplotlib import pyplot as plt
from keras.preprocessing import image

prediction = ""

test_image = image.load_img('1443.jpg',
                              target_size = (224, 224))

plt.imshow(test_image, 'gray')


test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = custom_Vgg16_Model.predict(test_image)

saida = [np.argmax(t) for t in result]

if saida == [0]:
    prediction = 'Benigno'
else:
    prediction = 'Maligno'

plt.title(prediction)    
plt.show() 